# DB Populate Notebook

This notebook populates the database with a variety of apps, records, and
feedback results. It is used primarily for database migration testing.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Enables: Debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
# !pip install llama_index==0.9.15.post2
# !pip install pydantic==2.5.2 pydantic_core==2.14.5

In [ ]:
# To test out DB migrations, copy one of the older db dumps to this folder first:

! ls ../../release_dbs/
! cp ../../release_dbs/0.3.0/default.sqlite default.sqlite
# ! rm default.sqlite

In [ ]:
from pathlib import Path

from trulens.core import Feedback
from trulens.core import Tru
from trulens.dashboard import run_dashboard
from trulens.providers.huggingface.provider import Dummy

In [ ]:
from trulens.core.database.migrations.data import (
    _sql_alchemy_serialization_asserts,
)

# Setup Tru and/or dashboard.

tru = Tru(database_redact_keys=True)

# tru.reset_database()

run_dashboard(tru, force=True, _dev=Path().cwd().parent.parent.resolve())

Tru().migrate_database()

_sql_alchemy_serialization_asserts(tru.db)

# Feedbacks

In [ ]:
# Dummy endpoint

dummy = Dummy(
    loading_prob=0.1,
    freeze_prob=0.0,  # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.01,
    overloaded_prob=0.1,
    rpm=6000,
)

f_lang_match_dummy = Feedback(dummy.language_match).on_input_output()

In [ ]:
# Huggingface endpoint
from trulens.providers.huggingface import Huggingface

hugs = Huggingface()

f_lang_match_hugs = Feedback(hugs.language_match).on_input_output()

In [ ]:
# import inspect
# inspect.signature(Huggingface).bind()

In [ ]:
# Openai endpoint
from trulens.providers.openai import OpenAI

openai = OpenAI()

f_relevance_openai = Feedback(openai.relevance).on_input_output()

In [ ]:
# Bedrock endpoint
# Cohere as endpoint
# LangChain as endpoint
# Litellm as endpoint

In [ ]:
feedbacks = [f_lang_match_hugs, f_lang_match_dummy, f_relevance_openai]

# _LangChain_ app

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from trulens.core.instruments import instrument
from trulens.instrument.langchain import TruChain

llm = OpenAI(temperature=0.9, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
)

# Conversational app puts it all together.
app_langchain = ConversationChain(llm=llm, memory=memory)


instrument.method(PromptTemplate, "format")

truchain = TruChain(app_langchain, app_id="langchain_app", feedbacks=feedbacks)

In [ ]:
with truchain as recs:
    print(app_langchain("Hello?"))

# Llama-index app

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from trulens.instrument.llamaindex import TruLlama

documents = SimpleWebPageReader(html_to_text=True).load_data([
    "http://paulgraham.com/worked.html"
])
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

trullama = TruLlama(query_engine, app_id="llama_index_app")

In [ ]:
with trullama as recs:
    print(query_engine.query("Who is the author?"))

# Basic app

In [ ]:
from trulens.core import TruBasic
from trulens.core.app.custom import instrument


def custom_application(prompt: str) -> str:
    return f"a useful response to {prompt}"


trubasic = TruBasic(custom_application, app_id="basic_app", feedbacks=feedbacks)

In [ ]:
with trubasic as recs:
    print(trubasic.app("hello?"))

# Custom app

In [ ]:
from trulens.core import TruCustomApp

from examples.dev.dummy_app.app import DummyApp

# Create custom app:
app_custom = DummyApp()

# Create trulens wrapper:
trucustom = TruCustomApp(
    app=app_custom,
    app_id="custom_app",
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app_custom.respond_to_query,
    feedbacks=feedbacks,
)

In [ ]:
with trucustom as recs:
    print(app_custom.respond_to_query("hello there"))